<center><H1>Modelado con RNN tipo <i>sequence to sequence</i> para la predicción de la demanda de energía eléctrica</H1><center>

<center><img src="https://www.gstatic.com/devrel-devsite/prod/ve2848ad92313fddfcd40baeb58a2f663fe2fd55c371a714a6bb3e329e2b15223/tensorflow/images/lockup.svg"  height="80px" style="padding-bottom:5px;"  /></center>

<center><H2>Julio Waissman Vilanova</H2>

<table align="center">
      <td align="center"><a target="_blank" href="https://www.unison.mx">
            <img src="https://www.unison.mx/wp-content/themes/awaken/images/logo.png"  height="70px" style="padding-bottom:5px;"  /></a></td>  
      <td align="center"><a target="_blank" href="https://www.gob.mx/cenace">
            <img src="https://universidad.cenace.gob.mx/pluginfile.php/244/block_html/content/CENACE-logo-completo.png" width="300" style="padding-bottom:5px;" /></a></td>
      <td align="center"><a target="_blank" href="https://colab.research.google.com/github/juliowaissman/rn-cenace/blob/main/encoders_GRNO.ipynb">
            <img src="https://i.ibb.co/2P3SLwK/colab.png"  style="padding-bottom:5px;" />Ejecuta en Google Colab</a></td>

</table>

In [4]:

# Las bibliotecas de base
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Para normalizar los datos de entrada
from sklearn.preprocessing import MinMaxScaler

#Tensorflow con keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Gráficas más fáciles de manipular con plotly
import plotly.express as px
import plotly.graph_objects as go

# Como se verán las gráficas de matplotlib
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (15,7)

## 1. Introducción

En esta libreta vamos a desarrollar pasito a pasito un modelo para la predicción en redes neuronales utilizando un modelo de secuencia a secuencia.

En un modelo de secuencia a secuencia, se utilizan básicamente dos redes, la primera recibe y procesa los datos de entrada (los cuales van a ser una serie de datos que entran a lo largo del tiempo). Una vez que se procesan los datos, estos se introducen en otra red (via un *vector de contexto*) para generar una secuencia de datos de estimación.

Básicamente esto se logra con una *repeat vector layer* entre la red neuronal de procesamiento de la entrada (el *encoder*) y la red de salida (el *decoder*). 

Vamos a tratar de hacerlo parte por parte de manera que todo quede lo más claro posible. Así que haremos todo paso a paso.

## 2. Descargar los datos

Empecemos por cargar los datos:

In [ ]:
url = "https://github.com/juliowaissman/curso-ml-cenace/raw/main/datos/Dataset_GCRNO_05052021.xlsx"

df = pd.read_excel(url, sheet_name='Datos')

df_horario = df.melt(
    id_vars= ['FECHA'],
    value_vars= [f'DEM_GCRNO_H{i}' for i in range(24)],
    var_name="Hora",
    value_name="Demanda"
).replace(
    {f'DEM_GCRNO_H{i}': i for i in range(24)}
)

df_horario.index = df_horario.FECHA + pd.to_timedelta(df_horario.Hora, unit='h')
df_horario.sort_index(inplace=True)
df_horario.drop(columns=['FECHA', 'Hora'], inplace=True)
df_horario = df_horario.asfreq('H', method='pad')
df_horario["Fecha"] = df_horario.index
df_horario["Dia"] = df_horario.index.weekday
df_horario["Hora"] = df_horario.index.hour
df_horario["Mes"] = df_horario.index.month

print(df_horario.info())

fig = px.line(df_horario, x="Fecha", y="Demanda", title='Demanda de energía GRNO')
fig.show()

## 3. Acondicionamiento de datos para el aprendizaje


Vamos a utilizar los datos del 2021 para prueba y el resto de entrenamiento

In [ ]:
df_train = df_horario[df_horario.index.year < 2020]
df_val = df_horario[df_horario.index.year == 2020]
df_test = df_horario[df_horario.index.year == 2021]

df_train.shape, df_val.shape, df_test.shape

Debido al algoritmo de B-prop a través del tiempo, las RNN son muy sensibles a los valores de los datos de entrada (en particular las LSTMs) por lo que es importante normalizar los datos. Notese que vamos a ajustar el `scaler` con los datos de entrenamiento, y se usa el mismo para ajustar los datos de prueba.

Vamos a generar los objetos `scaler` necesarios para escalar a todas las variables que estamos utilizando.

In [ ]:
train = df_train
scalers = {}  # Un diccionario con los scalers

for attr in ['Demanda', 'Mes', 'Dia', 'Hora']:
  scaler = MinMaxScaler(feature_range=(-1, 1))
  s_s = scaler.fit_transform(train[attr].values.reshape(-1,1))
  scalers[attr] = scaler
  train[attr] = s_s.ravel()

train.info()

y ahora vamos a escalar los datos de `val` y `test` con el `scaler` ajustado:

In [ ]:
val = df_val
for attr in ['Demanda', 'Mes', 'Dia', 'Hora']:
  scaler = scalers[attr]
  s_s = scaler.transform(val[attr].values.reshape(-1,1))
  val[attr] = s_s.ravel()

val.info()

In [ ]:
test = df_test
for attr in ['Demanda', 'Mes', 'Dia', 'Hora']:
  scaler = scalers[attr]
  s_s = scaler.transform(test[attr].values.reshape(-1,1))
  test[attr] = s_s.ravel()

test.info()

Pero esto no es suficiente para poder entrenar una red neuronal. Para poder entrenar la red necesitamos convertir los datos en muestras de entradas con observaciones pasada y con salidas futuras. para ser usados en el entrenamiento.

Vamos a hacer una función para esto:

In [21]:
def divide_series(series, n_pasado, n_futuro, n_salto, es_train=True):
  """
  n_pasado: número de observaciones pasadas para el encoder 
  n_futuro: número de observaciones futuras
  n_salto: a partir de donde empiezan a contar las observaciones futuras

  """
  X, y = list(), list() # Vamos a crear listas y al final hacemos ndarrays
  generador = range(len(series)) if es_train else range(0, len(series), n_futuro)
  
  for ini in generador:
    fin_anterior = ini + n_pasado
    fin_actual = fin_anterior + n_salto + n_futuro
    if fin_actual > len(series):
      break
    pasado = series[ini: fin_anterior, :]
    futuro = series[fin_anterior + n_salto: fin_actual, :]
    X.append(pasado)
    y.append(futuro)
  return np.array(X), np.array(y)

Para nuestro caso, vamos a usar (esto lo podemos cambiar después) 7 días de información pasada, vamos a estimar 24 horas futuras, y las vamos a estimar después de que pasen 12 horas.

In [22]:
n_pasado = 24 * 7
n_futuro = 24
n_salto = 12

y ahora vamos a generar nuestos conjuntos de aprendizaje:

In [ ]:
# Atributos a utilizar
nom_attr = ['Demanda', 'Mes', 'Dia', 'Hora']
n_attr = len(nom_attr)

X_train, y_train = divide_series(train[nom_attr].values, n_pasado, n_futuro, n_salto)
X_val, y_val = divide_series(val[nom_attr].values, n_pasado, n_futuro, n_salto)
X_test, y_test = divide_series(test[nom_attr].values, n_pasado, n_futuro, n_salto, es_train=False)


# Reacomodar como un tensor de 3 dimensiones
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], n_attr))
X_val = X_val.reshape((X_val.shape[0], X_val.shape[1], n_attr))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], n_attr))

y_train = y_train.reshape((y_train.shape[0], y_train.shape[1], n_attr))
y_val = y_val.reshape((y_val.shape[0], y_val.shape[1], n_attr))
y_test = y_test.reshape((y_test.shape[0], y_test.shape[1], n_attr))

X_train.shape, X_val.shape, X_test.shape, y_train.shape, y_val.shape, y_test.shape


## 4. Modelo neuronal

El modelo que vamos a usar está dividio en dos partes: el *encoder* y el *decoder*. Vamos iniciando definiendo el *encoder*:

In [24]:
encoder_inputs = layers.Input(shape=(n_pasado, n_attr))

encoder_l1 = layers.LSTM(100, return_state=True)
encoder_outputs1 = encoder_l1(encoder_inputs)

encoder_states1 = encoder_outputs1[1:]

y ahora al decoder:

In [25]:
decoder_rvec = layers.RepeatVector(n_futuro)
decoder_inputs = decoder_rvec(encoder_outputs1[0])

decoder_l1 = layers.LSTM(100, return_sequences=True)
decoder_l1_output = decoder_l1(decoder_inputs, initial_state=encoder_states1)

decoder_l2 = layers.TimeDistributed(layers.Dense(n_attr))
decoder_outputs = decoder_l2(decoder_l1_output)

y ahora juntamos los dos modelos (método funcional)

In [ ]:
modelo = keras.models.Model(encoder_inputs, decoder_outputs)
modelo.summary()

## 5 Entrenando el modelo

Vamos ahora a aprovechar para ver como se usa un método de calendarización de la tasa de aprendizaje


In [ ]:
reduce_lr = keras.callbacks.LearningRateScheduler(lambda x: 1e-3 * 0.90 ** x)

path_checkpoint = "model_checkpoint.h5"
modelckpt_callback = keras.callbacks.ModelCheckpoint(
    monitor="val_loss",
    filepath=path_checkpoint,
    verbose=1,
    save_weights_only=True,
    save_best_only=True,
)


es_callback = keras.callbacks.EarlyStopping(
    monitor="val_loss", 
    min_delta=0, 
    patience=5
)

modelo.compile(
    optimizer=keras.optimizers.Adam(), 
    loss="mae"
)

history = modelo.fit(
    X_train,
    y_train,
    epochs=25,
    validation_data=(X_val,y_val),
    batch_size=32,
    callbacks=[reduce_lr, es_callback, modelckpt_callback]
)

Si tuviste que parar el entrenamiento, o simplemente prefieres no gastar tu cuota de GPU, lo puedes cargar desde el checkpoint.

In [ ]:
!curl -O https://github.com/juliowaissman/rn-cenace/raw/main/model_checkpoint.h5

In [65]:
modelo.load_weights("drive/MyDrive/model_checkpoint_original.h5")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Vamos entonces a simular los valores de salida

In [ ]:
y_est = modelo.predict(X_test)
y_test[:,:,0].ravel().shape, y_est[:,:,0].ravel().shape, df_test.Fecha[n_pasado + n_salto:-12].shape

y ahora vamos a convertir ambas series en un data frame, y vamos a aplicarles el scaler al revés:

In [52]:
scaler = scalers['Demanda']

yr = scaler.inverse_transform(y_test[:,:,0].ravel().reshape(-1, 1))
yh = scaler.inverse_transform(y_est[:,:,0].ravel().reshape(-1, 1))

df_est = pd.DataFrame({
    "Real": yr.ravel(),
    "Estimado": yh.ravel(),
    "Fecha": df_test.Fecha[n_pasado + n_salto:-12]     
})

Y por último vamos a graficar la salida estimada con plotly

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_est.Fecha, y=df_est.Estimado, name="Estimada"))
fig.add_trace(go.Scatter(x=df_est.Fecha, y=df_est.Real, name="Real"))
fig.update_layout(title="Estimación de la demanda")
fig.show()

## Ejercicio

Prueba con una red más compleja, por ejemplo (pero no necesariamente) esta:

In [ ]:
encoder_inputs = layers.Input(shape=(n_pasado, n_attr))

#------------------------------------------------------------------------------

encoder_l1 = layers.GRU(100, return_sequences=True, return_state=True)
encoder_outputs1 = encoder_l1(encoder_inputs)
encoder_states1 = encoder_outputs1[1:]

encoder_l2 = layers.GRU(100, return_state=True)
encoder_outputs2 = encoder_l2(encoder_outputs1[0])
encoder_states2 = encoder_outputs2[1:]

#------------------------------------------------------------------------------

repeat_vector = layers.RepeatVector(n_attr)
decoder_inputs = repeat_vector(encoder_outputs2[0])

#-------------------------------------------------------------------------------

decoder_l1 = layers.GRU(100, return_sequences=True)
decoder_output1 = decoder_l1(decoder_inputs, initial_state=encoder_states1)


decoder_l2 = tf.keras.layers.GRU(100, return_sequences=True)
decoder_output2 = decoder_l2(decoder_output1, initial_state=encoder_states2)


decoder_l3 = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(n_attr))
decoder_outputs = decoder_l3(decoder_output2)

#-------------------------------------------------------------------------------

modelo2 = keras.models.Model(encoder_inputs, decoder_outputs)
modelo2.summary()

Aqui está el esquemático de éste modelo:

![](https://github.com/juliowaissman/rn-cenace/raw/main/modelo_alternativo.jpg)